In [ ]:
from Crawler.Utils import get_and_unwrap
from Crawler.Data import SyllabusRegistry
from Crawler.Constants import Constants

In [ ]:
html = get_and_unwrap(Constants.CMU_SYLLABUS_REGISTRY_URL.value)

In [ ]:
sr = SyllabusRegistry(html=html)

f24 = sr.semesters[0]
f24_archi = f24.departments[0]
f24_archi.get()
f24_archi_courses = f24_archi.courses

f24_archi_courses['Available Syllabi'][3].href
f24_archi_courses['Available Syllabi'][3].get()
f24_archi_courses['Available Syllabi'][3].html

## Sqlite

In [ ]:
import pickle
with open('syllabus_registry.pkl', 'rb') as f:
    sr = pickle.load(f) 

In [ ]:
import requests
from Crawler.Constants import Constants 
response = requests.get('https://canvas.cmu.edu/courses/40989/modules/items/5906263', cookies=Constants.COOKIE.value)
response = requests.get('https://canvas.cmu.edu/courses/5128', cookies=Constants.COOKIE.value)
response = requests.get('https://canvas.cmu.edu/courses/3760', cookies=Constants.COOKIE.value)